In [6]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 588.0 kB/s eta 0:00:00


In [17]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.2 MB/s eta 0:00:00


In [18]:
from datasets import get_dataset_config_names
get_dataset_config_names("common_voice")


['ab',
 'ar',
 'as',
 'br',
 'ca',
 'cnh',
 'cs',
 'cv',
 'cy',
 'de',
 'dv',
 'el',
 'en',
 'eo',
 'es',
 'et',
 'eu',
 'fa',
 'fi',
 'fr',
 'fy-NL',
 'ga-IE',
 'hi',
 'hsb',
 'hu',
 'ia',
 'id',
 'it',
 'ja',
 'ka',
 'kab',
 'ky',
 'lg',
 'lt',
 'lv',
 'mn',
 'mt',
 'nl',
 'or',
 'pa-IN',
 'pl',
 'pt',
 'rm-sursilv',
 'rm-vallader',
 'ro',
 'ru',
 'rw',
 'sah',
 'sl',
 'sv-SE',
 'ta',
 'th',
 'tr',
 'tt',
 'uk',
 'vi',
 'vot',
 'zh-CN',
 'zh-HK',
 'zh-TW']

In [20]:
import datasets
datasets.config.HF_DATASETS_CACHE = None

In [23]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch

# Load dataset
dataset = load_dataset("common_voice", "ar", split="train[:1%]", download_mode="force_redownload")

# Preprocessing
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53")

def preprocess(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    return batch

dataset = dataset.map(preprocess)

# Training config
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    logging_steps=10,
    save_steps=10,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()


Generating train split:   0%|          | 0/14227 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [24]:
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = torch.argmax(torch.tensor(pred_logits), dim=-1)
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    compute_metrics=compute_metrics
)


NameError: name 'model' is not defined